In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
# 读取CSV文件
BasicProp_path = os.path.join("..", "data", "HYPROP", "DataFiles_V2", "BasicProp.csv")
Param_path = os.path.join("..","data", "HYPROP", "DataFiles_V2","Param.csv")
MetaData_path = os.path.join("..", "data", "HYPROP", "DataFiles_V2", "MetaData.csv")
CondMeas_path = os.path.join("..", "data", "HYPROP", "DataFiles_V2", "CondMeas.csv")

In [2]:
# 处理MetaData.csv
known_columns0 = ["Sample_ID", "Lat", "Lon", "SamplingDepth"]
MetaData_data =pd.read_csv(MetaData_path)[known_columns0]
print(MetaData_data)

    Sample_ID        Lat     Lon  SamplingDepth
0    FSUJ_001  51.109000  10.412           27.5
1    FSUJ_002  51.109000  10.412           27.5
2    FSUJ_003  51.109000  10.413           27.5
3    FSUJ_004  51.108000  10.413           27.5
4    FSUJ_005  51.108000  10.413           27.5
..        ...        ...     ...            ...
567   UFZ_041  31.899722  34.850           12.5
568   UFZ_042  31.899722  34.850           12.5
569   UFZ_043  31.899722  34.850           12.5
570   UFZ_044  31.899722  34.850           12.5
571   UFZ_045  31.899722  34.850           12.5

[572 rows x 4 columns]


In [3]:
# 处理BasicProp.csv
BasicProp_data = pd.read_csv(BasicProp_path)
known_columns1 = ["Sample_ID","Sand_USDA","Silt_USDA", "Clay_USDA", "Corg", "BD", "Stone"]
known_columns2 = ["Sample_ID","Sand_USDA","Silt_USDA", "Clay_USDA", "Corg", "BD"]
known_columns3 = ["Sample_ID","Sand_USDA","Silt_USDA", "Clay_USDA", "Corg"]
known_columns4 = ["Sample_ID","Sand_USDA","Silt_USDA", "Clay_USDA"]
BasicProp_data = BasicProp_data[known_columns1]
print(BasicProp_data)

    Sample_ID  Sand_USDA  Silt_USDA  Clay_USDA  Corg    BD  Stone
0    FSUJ_001  15.884116  70.129870  13.986014  0.97  1.41    1.2
1    FSUJ_002  13.286713  67.632368  19.080919  1.08  1.32    0.2
2    FSUJ_003  10.000000  74.300000  15.700000  1.00  1.38    0.0
3    FSUJ_004   8.991009  69.530470  21.478521  1.35  1.44    0.1
4    FSUJ_005  13.200000  84.400000   2.400000  0.88  1.45    0.8
..        ...        ...        ...        ...   ...   ...    ...
567   UFZ_041  84.015984   5.394605  10.589411  1.41  1.36    NaN
568   UFZ_042  88.000000   4.500000   7.500000  1.14  1.46    NaN
569   UFZ_043  90.600000   3.400000   6.000000  1.28  1.11    NaN
570   UFZ_044  75.675676  10.010010  14.314314  2.41  1.10    NaN
571   UFZ_045  90.609391   3.696304   5.694306  0.92  1.29    NaN

[572 rows x 7 columns]


In [4]:
# 构建特征数据
merged_data_x =  pd.merge(MetaData_data, BasicProp_data, left_on='Sample_ID', right_on='Sample_ID')
print(merged_data_x)

    Sample_ID        Lat     Lon  SamplingDepth  Sand_USDA  Silt_USDA  \
0    FSUJ_001  51.109000  10.412           27.5  15.884116  70.129870   
1    FSUJ_002  51.109000  10.412           27.5  13.286713  67.632368   
2    FSUJ_003  51.109000  10.413           27.5  10.000000  74.300000   
3    FSUJ_004  51.108000  10.413           27.5   8.991009  69.530470   
4    FSUJ_005  51.108000  10.413           27.5  13.200000  84.400000   
..        ...        ...     ...            ...        ...        ...   
567   UFZ_041  31.899722  34.850           12.5  84.015984   5.394605   
568   UFZ_042  31.899722  34.850           12.5  88.000000   4.500000   
569   UFZ_043  31.899722  34.850           12.5  90.600000   3.400000   
570   UFZ_044  31.899722  34.850           12.5  75.675676  10.010010   
571   UFZ_045  31.899722  34.850           12.5  90.609391   3.696304   

     Clay_USDA  Corg    BD  Stone  
0    13.986014  0.97  1.41    1.2  
1    19.080919  1.08  1.32    0.2  
2    15.700000 

In [5]:
# 处理Param.csv
Param_data = pd.read_csv(Param_path).iloc[:, 0:5]
print(Param_data)

    Sample_ID  al_VGM  n_VGM  thr_VGM  ths_VGM
0    FSUJ_001  0.0114   1.23   0.0000   0.4314
1    FSUJ_002  0.0055   1.30   0.0000   0.4491
2    FSUJ_003  0.0068   1.29   0.0000   0.4407
3    FSUJ_004  0.0025   1.37   0.0000   0.3558
4    FSUJ_005  0.0065   1.28   0.0000   0.3991
..        ...     ...    ...      ...      ...
567   UFZ_041  0.0269   1.29   0.0000   0.4432
568   UFZ_042  0.0328   1.32   0.0000   0.4031
569   UFZ_043  0.0475   2.51   0.0842   0.4678
570   UFZ_044  0.0930   1.63   0.1256   0.4470
571   UFZ_045  0.0433   2.27   0.0749   0.3795

[572 rows x 5 columns]


In [6]:
# 处理CondMeas.csv
CondMeas_data= pd.read_csv(CondMeas_path)
ksat_rows = CondMeas_data[CondMeas_data['MeasType'] == 'KSAT'][['Sample_ID', 'k']]
print(ksat_rows)

      Sample_ID            k
0      FSUJ_001   416.869383
124    FSUJ_002  1174.897555
231    FSUJ_003   691.830971
327    FSUJ_004    70.794578
528    FSUJ_005   223.872114
...         ...          ...
41707  TUDD_031    38.904514
41809  TUDD_032    53.703180
44785   UFZ_040     6.165950
45142   UFZ_043     9.332543
45248   UFZ_045     6.165950

[409 rows x 2 columns]


In [7]:
# 构建目标数据
merged_data_y = pd.merge(Param_data, ksat_rows, left_on='Sample_ID', right_on='Sample_ID')
print(merged_data_y)

    Sample_ID  al_VGM  n_VGM  thr_VGM  ths_VGM            k
0    FSUJ_001  0.0114   1.23   0.0000   0.4314   416.869383
1    FSUJ_002  0.0055   1.30   0.0000   0.4491  1174.897555
2    FSUJ_003  0.0068   1.29   0.0000   0.4407   691.830971
3    FSUJ_004  0.0025   1.37   0.0000   0.3558    70.794578
4    FSUJ_005  0.0065   1.28   0.0000   0.3991   223.872114
..        ...     ...    ...      ...      ...          ...
404  TUDD_031  0.0030   1.31   0.0000   0.4624    38.904514
405  TUDD_032  0.0029   1.32   0.0000   0.4057    53.703180
406   UFZ_040  0.0214   2.19   0.1100   0.4194     6.165950
407   UFZ_043  0.0475   2.51   0.0842   0.4678     9.332543
408   UFZ_045  0.0433   2.27   0.0749   0.3795     6.165950

[409 rows x 6 columns]


In [8]:
# 获取merged_data_y中的Sample_ID列表
sample_ids_to_keep = merged_data_y['Sample_ID'].unique()
# 从merged_data_x中筛选出与merged_data_y中的Sample_ID匹配的行
filtered_merged_data_x = merged_data_x[merged_data_x['Sample_ID'].isin(sample_ids_to_keep)]
# 重置索引以使序号从0开始
filtered_merged_data_x = filtered_merged_data_x.reset_index(drop=True)
print(filtered_merged_data_x)

    Sample_ID        Lat        Lon  SamplingDepth  Sand_USDA  Silt_USDA  \
0    FSUJ_001  51.109000  10.412000           27.5  15.884116  70.129870   
1    FSUJ_002  51.109000  10.412000           27.5  13.286713  67.632368   
2    FSUJ_003  51.109000  10.413000           27.5  10.000000  74.300000   
3    FSUJ_004  51.108000  10.413000           27.5   8.991009  69.530470   
4    FSUJ_005  51.108000  10.413000           27.5  13.200000  84.400000   
..        ...        ...        ...            ...        ...        ...   
404  TUDD_031  51.118333  13.228611            2.5   6.100000  73.700000   
405  TUDD_032  51.118333  13.228611            2.5   8.100000  73.500000   
406   UFZ_040  31.899722  34.850000           12.5  91.608392   2.197802   
407   UFZ_043  31.899722  34.850000           12.5  90.600000   3.400000   
408   UFZ_045  31.899722  34.850000           12.5  90.609391   3.696304   

     Clay_USDA  Corg    BD  Stone  
0    13.986014  0.97  1.41    1.2  
1    19.080919 

In [9]:
# 样品编号列不需要，需要删除
filtered_merged_data_x = filtered_merged_data_x .iloc[:, 1:]
merged_data_y = merged_data_y.iloc[:, 1:]
# 使用均值填充NaN值
filtered_merged_data_x = filtered_merged_data_x.fillna(filtered_merged_data_x.mean())
merged_data_y = merged_data_y.fillna(merged_data_y.mean())

随机森林方法构建PTFs

In [10]:
# 数据预处理
X = filtered_merged_data_x.values  # 使用所有列作为特征
y = merged_data_y.values  # 使用所有列作为目标

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  # 将数据划分为训练集和测试集

# 创建并训练模型
model = RandomForestRegressor(n_estimators=1000, random_state=10)  # 实例化随机森林回归模型
model.fit(X_train, y_train)  # 使用训练数据拟合模型

# 预测
y_pred = model.predict(X_test)  # 使用测试数据进行预测

# 计算预测误差
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')  # 计算均方误差
r2 = r2_score(y_test, y_pred, multioutput='raw_values')  # 计算R²分数
print(f"Mean Squared Error for each output: {mse, r2}")  # 输出每个输出的均方误差和R²分数

Mean Squared Error for each output: (array([3.18944531e-03, 8.00522566e-02, 3.93727800e-04, 1.48972689e-03,
       9.40804577e+06]), array([0.31549538, 0.86964252, 0.24406272, 0.82253121, 0.24158795]))


神经网络方法构建PTFs

In [11]:
# 数据预处理
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(filtered_merged_data_x.values)  # 对X进行标准化
y = scaler_y.fit_transform(merged_data_y.values)  # 对y进行标准化

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 定义神经网络结构
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # 定义第一个全连接层
        self.bn1 = nn.BatchNorm1d(hidden_size)  # 添加批量归一化层
        self.fc2 = nn.Linear(hidden_size, output_size)  # 定义第二个全连接层

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # 通过激活函数和批量归一化层传递输入
        x = self.fc2(x)  # 通过第二个全连接层传递输入
        return x

# 创建模型、损失函数和优化器
input_size = X_train.shape[1]
hidden_size = 64
output_size = y_train.shape[1]
model = Net(input_size, hidden_size, output_size)  # 实例化神经网络
criterion = nn.MSELoss()  # 定义均方误差损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # 定义Adam优化器并添加权重衰减

# 训练模型
epochs = 5000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()  # 清除梯度
    y_pred = model(torch.tensor(X_train, dtype=torch.float32))  # 通过模型进行预测
    loss = criterion(y_pred, torch.tensor(y_train, dtype=torch.float32))  # 计算损失
    loss.backward()  # 计算梯度
    optimizer.step()  # 更新权重
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

# 预测
model.eval()
with torch.no_grad():
    y_pred = model(torch.tensor(X_test, dtype=torch.float32))  # 在测试集上进行预测

# 反标准化预测值和真实值
y_pred = scaler_y.inverse_transform(y_pred.numpy())  # 将预测值反标准化
y_test = scaler_y.inverse_transform(y_test)  # 将真实值反标准化

# 计算预测误差
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')  # 计算均方误差
r2 = r2_score(y_test, y_pred, multioutput='raw_values')  # 计算R²分数
print(f"Mean Squared Error for each output: {mse, r2}")

Epoch 1/5000, Loss: 1.1812852621078491
Epoch 2/5000, Loss: 1.148480772972107
Epoch 3/5000, Loss: 1.1171826124191284
Epoch 4/5000, Loss: 1.0873303413391113
Epoch 5/5000, Loss: 1.0589947700500488
Epoch 6/5000, Loss: 1.0321327447891235
Epoch 7/5000, Loss: 1.006614089012146
Epoch 8/5000, Loss: 0.9823979139328003
Epoch 9/5000, Loss: 0.9594271779060364
Epoch 10/5000, Loss: 0.9376528859138489
Epoch 11/5000, Loss: 0.9170388579368591
Epoch 12/5000, Loss: 0.8975399136543274
Epoch 13/5000, Loss: 0.8790817856788635
Epoch 14/5000, Loss: 0.8615881204605103
Epoch 15/5000, Loss: 0.8449921011924744
Epoch 16/5000, Loss: 0.829281210899353
Epoch 17/5000, Loss: 0.8144146203994751
Epoch 18/5000, Loss: 0.8003209829330444
Epoch 19/5000, Loss: 0.7869877815246582
Epoch 20/5000, Loss: 0.7743424773216248
Epoch 21/5000, Loss: 0.7623337507247925
Epoch 22/5000, Loss: 0.750889003276825
Epoch 23/5000, Loss: 0.7399709820747375
Epoch 24/5000, Loss: 0.7295854091644287
Epoch 25/5000, Loss: 0.7196771502494812
Epoch 26/5000